
# GP2GP Technical Error Counts - Feb, Sept and Oct 2020

Context:

JGPIT Futures would like to know the current impact/size of the problem of paper processing as a result of GP2GP failures. 
Finding the totals of technical failures involves calculating, for each month:
- the total number of transfers
- the total number of successful transfers completed within 8 days SLA
- the total number of successful transfers completed beyond 8 days SLA
- the total number of technical errors
- the total number of other process/tech errors

Assumptions:

We defined technical errors as:
- conversations that have completed but there is an error code in the final Application Acknowledgement Message MCCI_IN010000UK13 of the RCMR_IN030000UK06 Request Started message.
- conversations that are still pending (i.e. no final Application Acknowledgement Message MCCI_IN010000UK13 of the RCMR_IN030000UK06 Request Started message received within the following month), and have error codes in any of the intermediate messages (for example any other MCCI_IN010000UK13 Application Acknowledgement  Messages.

We defined other process/tech errors as:
- As conversations that are still pending (i.e. there is no final Application Acknowledgement Message MCCI_IN010000UK13 of the RCMR_IN030000UK06 Request Started message received within the following month) and without any errors

Requirements:

This notebook uses the following Splunk query, where we obtained data from entire month of Feburary 2020, March 2020, September 2020 and October 2020 and from the 1st to 25th November 2020:
```
index="spine2vfmmonitor" service="gp2gp"
| search interactionID="urn:nhs:names:services:gp2gp/*"
| rex field=fromPartyID "(?<fromNACS>.+)(-\d*)"
| rex field=toPartyID "(?<toNACS>.+)(-\d*)"
| fields _time, conversationID, GUID, interactionID, fromNACS, toNACS, messageRef, jdiEvent
| fields - _raw
```

In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import paths
from datetime import datetime
from dateutil.tz import tzutc

from gp2gp.date.range import DateTimeRange
from scripts.gp2gp_spine_outcomes import calculate_counts


In [4]:
february_data_file_name="../data/months/Feb-2020.csv.gz"
march_data_file_name = "../data/months/Mar-2020.csv.gz"
september_data_file_name="../data/months/Sept-2020.csv.gz"
october_data_file_name = "../data/months/Oct-2020.csv.gz"
november_data_file_name = "../data/months/Nov-2020.csv.gz"

In [5]:
february_time_range = DateTimeRange(
    datetime(year=2020, month=2, day=1, tzinfo=tzutc()),
    datetime(year=2020, month=3, day=1, tzinfo=tzutc()),
)

february_transfer_outcomes = calculate_counts(february_data_file_name, march_data_file_name, february_time_range)

In [6]:
september_time_range = DateTimeRange(
    datetime(year=2020, month=9, day=1, tzinfo=tzutc()),
    datetime(year=2020, month=10, day=1, tzinfo=tzutc()),
)

september_transfer_outcomes = calculate_counts(september_data_file_name, october_data_file_name, september_time_range)

In [9]:
october_time_range = DateTimeRange(
    datetime(year=2020, month=10, day=1, tzinfo=tzutc()),
    datetime(year=2020, month=11, day=1, tzinfo=tzutc()),
)

october_transfer_outcomes = calculate_counts(october_data_file_name, november_data_file_name, october_time_range)

In [12]:
transfer_outcomes_df = pd.DataFrame.from_dict([february_transfer_outcomes, september_transfer_outcomes, october_transfer_outcomes])
transfer_outcomes_df["months"] = ["February", "September", "October"]
transfer_outcomes_df = transfer_outcomes_df.set_index("months")
transfer_outcomes_df

,COMPLETED - WITHIN 8 DAYS,DIDN'T COMPLETE - ERROR MID CONVERSATION,DIDN'T COMPLETE - STUCK,COMPLETED - ERROR IN FINAL ACK,COMPLETED - BEYOND 8 DAYS
months,,,,,
February,175517,4384,6329,3559,10097
September,236905,5488,8096,5737,17439
October,205238,4847,6826,4837,12668


In [13]:
transfer_outcomes_df["TECHNICAL ERRORS"] = transfer_outcomes_df["DIDN'T COMPLETE - ERROR MID CONVERSATION"] + transfer_outcomes_df["COMPLETED - ERROR IN FINAL ACK"]
del transfer_outcomes_df["DIDN'T COMPLETE - ERROR MID CONVERSATION"]
del transfer_outcomes_df["COMPLETED - ERROR IN FINAL ACK"]

transfer_outcomes_df["TOTAL TRANSFERS"] = transfer_outcomes_df.sum(axis=1)

transfer_outcomes_df["TOTAL INTEGRATED"] = transfer_outcomes_df["COMPLETED - WITHIN 8 DAYS"] + transfer_outcomes_df["COMPLETED - BEYOND 8 DAYS"]

transfer_outcomes_df["PAPER FALLBACK"] = transfer_outcomes_df["COMPLETED - BEYOND 8 DAYS"] + transfer_outcomes_df["TECHNICAL ERRORS"] + transfer_outcomes_df["DIDN'T COMPLETE - STUCK"]

transfer_outcomes_df.rename(columns={"COMPLETED - WITHIN 8 DAYS": "INTEGRATED WITHIN 8 DAYS", "COMPLETED - BEYOND 8 DAYS": "INTEGRATED BEYOND 8 DAYS", "DIDN'T COMPLETE - STUCK": "OTHER PROCESS/TECH ERRORS"}, inplace=True)

transfer_outcomes_df = transfer_outcomes_df[["TOTAL TRANSFERS", "TOTAL INTEGRATED", "INTEGRATED WITHIN 8 DAYS", "PAPER FALLBACK", "INTEGRATED BEYOND 8 DAYS", "TECHNICAL ERRORS", "OTHER PROCESS/TECH ERRORS"]]

transfer_outcomes_df

,TOTAL TRANSFERS,TOTAL INTEGRATED,INTEGRATED WITHIN 8 DAYS,PAPER FALLBACK,INTEGRATED BEYOND 8 DAYS,TECHNICAL ERRORS,OTHER PROCESS/TECH ERRORS
months,,,,,,,
February,199886,185614,175517,24369,10097,7943,6329
September,273665,254344,236905,36760,17439,11225,8096
October,234416,217906,205238,29178,12668,9684,6826


In [34]:
outcome_percentages_df = transfer_outcomes_df.iloc[:, 1:].apply(lambda x: x / transfer_outcomes_df.iloc[:, 0] * 100)

outcome_percentages_df = outcome_percentages_df.add_suffix(' (%)')

outcome_percentages_df.round(2)

,TOTAL INTEGRATED (%),INTEGRATED WITHIN 8 DAYS (%),PAPER FALLBACK (%),INTEGRATED BEYOND 8 DAYS (%),TECHNICAL ERRORS (%),OTHER PROCESS/TECH ERRORS (%)
months,,,,,,
February,92.86,87.81,12.19,5.05,3.97,3.17
September,92.94,86.57,13.43,6.37,4.10,2.96
October,92.96,87.55,12.45,5.40,4.13,2.91
